<a href="https://colab.research.google.com/github/michalisgalanis/garbage-detection-internship/blob/main/Garbage_Detection_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Pre-processing**

## Acquiring the TACO dataset
TACO is a growing image dataset of waste in the wild. It contains images of litter taken under diverse environments: woods, roads and beaches. These images are manually labeled and segmented according to a hierarchical taxonomy to train and evaluate object detection algorithms. Currently, images are hosted on
Flickr and we have a server that is collecting more images and annotations.

Get more information [here](https://github.com/pedropro/TACO).

In [ ]:
# Installing required libraries
!pip install pillow
!pip install requests
!pip install jupyter
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install graphviz
!pip install Cython

In [ ]:
# Clone & Download dataset
import os

!git clone https://github.com/pedropro/TACO
os.chdir('/content/TACO')
!python3 download.py

## Organize files & folders

In [24]:
import os
import shutil

root_path = 'TACO/'
data_path = root_path + 'data/'
image_path = data_path + 'images/'

# Create new folder 'images'
os.mkdir(image_path)

# move all images from  multiple batch folders to a new single folder.
for batch in os.listdir(data_path):
    if os.path.isdir(data_path) and 'batch' in batch:
        for image in os.listdir(data_path + batch):
            shutil.move(data_path + batch + '/' + image, image_path + image)
        shutil.rmtree(data_path + batch)

## Load & Visualize annotations

The TACO dataset information is represented in a COCO format. To select and show dataset images with respective masks, we make use of the COCO API.

This sections uses the annotations.json file to generate a mask for every image in this dataset.

In [ ]:
from PIL import Image, ExifTags
from pycocotools.coco import COCO
from matplotlib.patches import Polygon, Rectangle
from matplotlib.collections import PatchCollection
import matplotlib.pyplot as plt
import numpy as np
import colorsys
import imageio
import shutil
import random
import pylab
import json
import os

# Paths
root_path = 'TACO/'
data_path = root_path + 'data/'
image_path = data_path + 'images/'
label_path = data_path + 'labels/'
anns_file_path = data_path + 'annotations.json'
pylab.rcParams['figure.figsize'] = (28,28)

# Read annotations
with open(anns_file_path, 'r') as f:
    dataset = json.loads(f.read())
categories = dataset['categories']
anns = dataset['annotations']
imgs = dataset['images']

# Clearing garbage (pun intented)
if (os.path.exists(label_path)): 
    shutil.rmtree(label_path)

# Create mask directory
os.mkdir(label_path)

# Generate mask for every image in this dataset
for image_filepath in os.listdir(image_path):
    # Obtain Exif orientation tag code
    for orientation in ExifTags.TAGS.keys():
        if ExifTags.TAGS[orientation] == 'Orientation':
            break

    # Loads dataset as a coco object
    coco = COCO(anns_file_path)

    # Find image id
    img_id = -1
    for img in imgs:
        check_image = img['file_name'].rpartition('/')[2]
        if check_image == image_filepath:
            img_id = img['id']
            break

    # Show image and corresponding annotations
    if img_id == -1:
        print('Incorrect file name')
    else:
        # Create binary mask from annotations.json and export as image
        anns_img = np.zeros((img['height'],img['width']))
        for ann in anns:
            if ann['image_id'] == img_id:
                a = coco.annToMask(ann)*ann['category_id']
                anns_img = np.maximum(anns_img,a)

        imageio.imwrite(label_path + image_filepath, anns_img)


# **Training**

In [6]:
import os, shutil

model_path = '/content/Mask_RCNN'
if os.path.exists(model_path): shutil.rmtree(model_path)
!git clone https://github.com/matterport/Mask_RCNN /content/Mask_RCNN

Cloning into '/content/Mask_RCNN'...
remote: Enumerating objects: 956, done.
remote: Total 956 (delta 0), reused 0 (delta 0), pack-reused 956
Receiving objects: 100% (956/956), 111.85 MiB | 6.47 MiB/s, done.
Resolving deltas: 100% (563/563), done.


In [ ]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt


ROOT_DIR = os.path.abspath("../content/Mask")




---



In [14]:
!cd /content/Mask_RCNN/
!python3 samples/coco/coco.py train --dataset=/content/TACO/data --model=coco

python3: can't open file 'samples/coco/coco.py': [Errno 2] No such file or directory


# **Testing**